## Importing Libraries

In [2]:
# %pip install pandas numpy matplotlib seaborn joblib scikit-learn xgboost lightgbm catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib as joblib
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
# %pip install xgboost;
# %pip install lightgbm;
# %pip install catboost;
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


## Load Data

In [3]:
data = pd.read_csv("../data/processed/upi_fraud_feature_engineered.csv")
X = data.drop(columns=['FraudFlag'])
y = data['FraudFlag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Training & Evaluate Models

In [4]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Support Vector Machine": SVC(probability=True),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0)
}

results = []

for name, model in models.items():
    pipeline = Pipeline([("scaler", StandardScaler()), ("model", model)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None

    results.append([name, accuracy, precision, recall, f1, roc_auc])

    joblib.dump(pipeline, f"../models/{name.replace(' ', '_').lower()}.pkl")

    print(f"\n{name} Model")
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))


Logistic Regression Model
Accuracy: 0.7774, Precision: 0.6234, Recall: 0.0886, F1-score: 0.1551, ROC-AUC: 0.6986

Confusion Matrix:
[[1778   29]
 [ 494   48]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.98      0.87      1807
           1       0.62      0.09      0.16       542

    accuracy                           0.78      2349
   macro avg       0.70      0.54      0.51      2349
weighted avg       0.75      0.78      0.71      2349


Decision Tree Model
Accuracy: 0.7944, Precision: 0.5492, Recall: 0.6070, F1-score: 0.5767, ROC-AUC: 0.7288

Confusion Matrix:
[[1537  270]
 [ 213  329]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.85      0.86      1807
           1       0.55      0.61      0.58       542

    accuracy                           0.79      2349
   macro avg       0.71      0.73      0.72      2349
weighted avg       0.80      0.79      

C:\Users\pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:158: UserWarning: [21:04:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



XGBoost Model
Accuracy: 0.9132, Precision: 0.9801, Recall: 0.6365, F1-score: 0.7718, ROC-AUC: 0.8435

Confusion Matrix:
[[1800    7]
 [ 197  345]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1807
           1       0.98      0.64      0.77       542

    accuracy                           0.91      2349
   macro avg       0.94      0.82      0.86      2349
weighted avg       0.92      0.91      0.91      2349

[LightGBM] [Info] Number of positive: 2168, number of negative: 7227
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3601
[LightGBM] [Info] Number of data points in the train set: 9395, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.230761 -> initscore=-1.204019
[LightGBM] [Info] Start training from score -1.204019

LightGBM 

C:\Users\pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\pavan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



CatBoost Model
Accuracy: 0.9166, Precision: 1.0000, Recall: 0.6384, F1-score: 0.7793, ROC-AUC: 0.8341

Confusion Matrix:
[[1807    0]
 [ 196  346]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1807
           1       1.00      0.64      0.78       542

    accuracy                           0.92      2349
   macro avg       0.95      0.82      0.86      2349
weighted avg       0.92      0.92      0.91      2349



In [5]:
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-score", "ROC-AUC"])
results_df.sort_values(by="Accuracy", ascending=False, inplace=True)
print("\nModel Performance Summary:\n", results_df)


Model Performance Summary:
                     Model  Accuracy  Precision    Recall  F1-score   ROC-AUC
8                CatBoost  0.916560   1.000000  0.638376  0.779279  0.834115
6                 XGBoost  0.913155   0.980114  0.636531  0.771812  0.843512
7                LightGBM  0.911877   0.997033  0.619926  0.764505  0.827659
3       Gradient Boosting  0.901660   1.000000  0.573801  0.729191  0.824564
2           Random Forest  0.873989   0.980469  0.463100  0.629073  0.872977
4  Support Vector Machine  0.841635   0.832031  0.392989  0.533835  0.818517
1           Decision Tree  0.794381   0.549249  0.607011  0.576687  0.728796
5     K-Nearest Neighbors  0.782035   0.520161  0.714022  0.601866  0.829211
0     Logistic Regression  0.777352   0.623377  0.088561  0.155089  0.698616


## Select Model

In [6]:
best_model_name = results_df.iloc[0]["Model"]
best_model = models[best_model_name]

print(f"Best Model Selected: {best_model_name}")

Best Model Selected: CatBoost


## Saving Model

In [7]:
import joblib
joblib.dump(best_model, f"../models/{best_model_name}.pkl")

['../models/CatBoost.pkl']